**Общая информация**

**Срок сдачи:** 18 октября 2017, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 18 октября, -4 балла после 06:00 25 октября, -6 баллов после 06:00 1 ноября

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack создать таск в приватный чат:
/todo Фамилия Имя ссылка на гитхаб @alkhamush
Пример:
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush
Дополнительно нужно просто скинуть ссылку в slack в личный чат

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

###### Задание 2 (2 балла)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy

###### Задание 3 (2 балла)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 4 (3 балла)
Добавить algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Значение n_neighbors нужно взять из задания 3. Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

###### Задание 5 (1 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)

**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять структуру кода и текст. В противном случае -1 балл


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

In [144]:
#%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase,
                             KNeighborsMixin,
                             SupervisedIntegerMixin,
                             ClassifierMixin):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        
        self._init_params(n_neighbors=n_neighbors,
                          algorithm=algorithm)
        
    def fit(self, X, y):
        self._X_train = X
        self._y_train = y
        
    def _predict(self, X):
        tmp = np.linalg.norm(self._X_train-X, axis=1)
        mask = np.argpartition(tmp, self.n_neighbors)[:self.n_neighbors]
        return np.bincount(self._y_train[mask]).argmax()
        
    def predict(self, X):
        if len(X.shape) == 1:
            return np.array([self._predict(X)])
        else:
            return np.apply_along_axis(self._predict, 1, X)

    def predict_proba(self, X):
        pass

    def score(self, X, y):
        return np.average(y == self.predict(X))

**IRIS**

In [145]:
iris = datasets.load_iris()

In [146]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [147]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [148]:
%time clf.fit(X_train, y_train)

CPU times: user 1.22 ms, sys: 347 µs, total: 1.57 ms
Wall time: 1.36 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [149]:
%time my_clf.fit(X_train, y_train)

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 18.1 µs


In [150]:
%time clf.predict(X_test)

CPU times: user 2.22 ms, sys: 1.13 ms, total: 3.35 ms
Wall time: 2.28 ms


array([1, 1, 0, 0, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 2])

In [151]:
%time my_clf.predict(X_test)

CPU times: user 3.15 ms, sys: 1.87 ms, total: 5.01 ms
Wall time: 3.25 ms


array([1, 1, 0, 0, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 2])

In [104]:
%time clf.predict_proba(X_test)

CPU times: user 1.1 ms, sys: 382 µs, total: 1.48 ms
Wall time: 1.15 ms


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

In [ ]:
%time my_clf.predict_proba(X_test)

In [152]:
clf.score(X_test, y_test)

0.8666666666666667

In [153]:
my_clf.score(X_test, y_test)

0.8666666666666667

Задание 3

**MNIST**

In [154]:
mnist = fetch_mldata('MNIST original')

In [155]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [156]:
clf = KNeighborsClassifier(n_neighbors=2)
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [157]:
%time clf.fit(X_train, y_train)

CPU times: user 43.5 s, sys: 808 ms, total: 44.3 s
Wall time: 45.4 s


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [158]:
%time my_clf.fit(X_train, y_train)

CPU times: user 14 µs, sys: 36 µs, total: 50 µs
Wall time: 55.8 µs


In [51]:
%time clf.predict(X_test)

CPU times: user 1min 9s, sys: 181 ms, total: 1min 9s
Wall time: 1min 9s


array([ 0.,  9.,  9.,  3.,  2.,  5.,  1.,  7.,  6.,  1.,  6.,  1.,  4.,
        2.,  4.,  4.,  1.,  5.,  1.,  4.,  3.,  2.,  8.,  4.,  8.,  1.,
        9.,  9.,  0.,  3.,  7.,  2.,  2.,  3.,  1.,  0.,  3.,  1.,  0.,
        6.,  9.,  7.,  8.,  3.,  1.,  5.,  4.,  3.,  8.,  5.,  2.,  2.,
        7.,  8.,  1.,  6.,  8.,  9.,  7.,  5.,  9.,  2.,  5.,  2.,  1.,
        3.,  6.,  3.,  3.,  0.,  9.,  9.,  3.,  6.,  4.,  0.,  3.,  0.,
        3.,  4.,  4.,  8.,  0.,  5.,  3.,  0.,  3.,  8.,  8.,  8.,  8.,
        7.,  5.,  9.,  7.,  6.,  6.,  4.,  0.,  9.,  4.,  0.,  2.,  0.,
        1.,  0.,  3.,  2.,  4.,  7.,  7.,  6.,  8.,  6.,  4.,  1.,  6.,
        1.,  3.,  1.,  2.,  1.,  8.,  4.,  2.,  9.,  7.,  4.,  7.,  6.,
        8.,  6.,  7.,  7.,  5.,  0.,  4.,  8.,  1.,  0.,  8.,  3.,  1.,
        1.,  0.,  2.,  6.,  5.,  7.,  8.,  3.,  5.,  4.,  7.,  2.,  2.,
        4.,  8.,  6.,  6.,  7.,  4.,  2.,  7.,  3.,  7.,  5.,  1.,  4.,
        4.,  6.,  8.,  1.,  8.,  6.,  8.,  3.,  9.,  5.,  4.,  0

In [128]:
%time my_clf.predict(X_test)

CPU times: user 4min 21s, sys: 4min 41s, total: 9min 2s
Wall time: 9min 7s


array([ 1.,  6.,  0.,  1.,  0.,  1.,  1.,  6.,  0.,  1.,  2.,  4.,  1.,
        3.,  3.,  1.,  2.,  4.,  1.,  1.,  1.,  1.,  2.,  0.,  5.,  4.,
        3.,  1.,  4.,  2.,  6.,  4.,  4.,  1.,  6.,  5.,  2.,  0.,  7.,
        4.,  3.,  1.,  1.,  4.,  9.,  1.,  1.,  1.,  1.,  4.,  7.,  1.,
        1.,  1.,  6.,  1.,  0.,  4.,  1.,  7.,  2.,  3.,  7.,  4.,  2.,
        1.,  5.,  1.,  2.,  4.,  9.,  2.,  1.,  7.,  2.,  7.,  1.,  7.,
        1.,  7.,  1.,  5.,  3.,  5.,  0.,  8.,  1.,  5.,  4.,  6.,  8.,
        0.,  5.,  1.,  1.,  3.,  1.,  6.,  0.,  4.,  4.,  7.,  8.,  1.,
        9.,  0.,  1.,  8.,  1.,  1.,  1.,  0.,  4.,  4.,  6.,  0.,  4.,
        4.,  4.,  1.,  1.,  7.,  4.,  4.,  1.,  7.,  2.,  1.,  1.,  4.,
        1.,  1.,  3.,  1.,  2.,  9.,  1.,  1.,  0.,  1.,  2.,  2.,  1.,
        5.,  1.,  4.,  5.,  6.,  0.,  1.,  2.,  4.,  7.,  3.,  2.,  1.,
        1.,  6.,  7.,  3.,  1.,  1.,  7.,  0.,  0.,  1.,  0.,  1.,  4.,
        2.,  5.,  1.,  4.,  1.,  3.,  3.,  8.,  2.,  1.,  7.,  7

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [53]:
clf.score(X_test, y_test)

0.96571428571428575

In [54]:
my_clf.score(X_test, y_test)

0.3914285714285714

Задание 4

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn
# matplotlib
# train_test_split
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline)
# StandardScaler (from sklearn.preprocessing import StandardScaler)
# ClassifierMixin
# NeighborsBase
# KNeighborsMixin
# SupervisedIntegerMixin